In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.inspection import PartialDependenceDisplay
from lightgbm import LGBMClassifier
import warnings, gc, string, random
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-may-2022/train.csv', index_col='id')
test=pd.read_csv('../input/tabular-playground-series-may-2022/test.csv', index_col='id')
sub=pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")


In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test

In [ ]:
train

In [ ]:
train.head()


In [ ]:
df=train.describe()

In [ ]:
df.style.format("{:,.3f}").background_gradient(subset=(df.index[1:],df.columns[:]),\
                                                             cmap="GnBu")
                                    

In [ ]:
print("Train Data:rows{:,.0f},columns{:,.0f},\n missingvalues {},Depulicates {}.\n".
      format(train.shape[0],train.shape[1], train.isna().sum().sum(), train.duplicated().sum()))

print("Test Data:rows{:,.0f},columns{:,.0f},\n missingvalues {},Depulicates {}.\n".
      format(test.shape[0],test.shape[1], test.isna().sum().sum(), test.duplicated().sum()))
    

## EDA

In [ ]:
train.target.value_counts(normalize=True)[::-1]
#0,1 => 1,0

So, when you do a[::-1], it starts from the end towards the first taking each element. So it reverses a. This is applicable for lists/tuples as well.
Example -
>>> a = '1234'\
>>> a[::-1]\
'4321'\

In [ ]:
temp=dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), 
                           height=500, width=1000))
target=train.target.value_counts(normalize=True)[::-1]
text=['State {}'.format(i) for i in target.index]

target=train.target.value_counts(normalize=True)[::-1]
test=["State{}".format(i) for i in target.index]
color,pal=['#18A6A5','#E1B590'],['#78CAC9','#EDD3B5']

if text[0]=='State 0':
    color,pal=color,pal
else:
    color,pal=color[::-1],pal[::-1]
fig=go.Figure()
fig.add_trace(go.Pie(labels=target.index, values=target*100, hole=.5, 
                     text=text, sort=False, showlegend=False,
                     marker=dict(colors=pal,line=dict(color=color,width=2)),
                     hovertemplate = "State %{label}: %{value:.2f}%<extra></extra>"))

fig.update_layout(template=temp, title='Target Distribution', 
                  uniformtext_minsize=15, uniformtext_mode='hide',width=700)

fig.show()
    


Customizing hover text with a hovertemplate\
https://plotly.com/python/hover-text-and-formatting/ 
To customize the tooltip on your graph you can use the hovertemplate attribute of graph_objects tracces, which is a template string used for rendering the information that appear on hoverbox. This template string can include variables in %{variable} format, numbers in d3-format's syntax, and date in d3-time-format's syntax. In the example below, the empty <extra></extra> tag removes the part of the hover where the trace name is usually displayed in a contrasting color. The <extra> tag can be used to display other parts of the hovertemplate, it is not reserved for the trace name.

Note that a hovertemplate customizes the tooltip text, while a texttemplate customizes the text that appears on your chart.

Set the horizontal alignment of the text within tooltip with hoverlabel.align.

## EDA Numerical value

In [ ]:
[colum for colum in train.columns][1:32][0]

In [ ]:
# Creating plot
# for i in range(0,32):
#     print(i)
#     index=[colum for colum in train.columns][1:32][i]
#     plt.boxplot(Feature train[index])
#     plt.show()
    
# show plot


In [ ]:
column=train.columns[1:32][0]
boxplot = train.boxplot(column=column)  

corelation

In [ ]:
corr=train.corr()
corr

In [ ]:
corr=corr.iloc[:-1,-1].sort_values(ascending=False)
corr

#remove target

In [ ]:
# corr_feature=[corr.index[:10][i] for i in range(0,10) ]
# train_df=train[corr_feature]
# train_df

In [ ]:
df

## Feature Eng

In [ ]:
df=train.copy()
df

In [ ]:
df['f_27']

In [ ]:
#count unique value
df['f_27'].apply(lambda x: len(set(x)))

In [ ]:
#Feature 27 consists of a string of 10 characters. 
df.f_27.str.len()

#What does -1 mean in numpy reshape? https://discuss.codingblocks.com/t/what-is-meaning-of-reshape-1-1/14830 

#get.value https://www.programiz.com/python-programming/methods/dictionary/get#:~:text=The%20get()%20method%20returns,key%20is%20in%20the%20dictionary.



In [ ]:
#OrdinalEncoder¶　https://pystyle.info/ml-encode-category-variable/#:~:text=OrdinalEncoder%20%E3%81%AF%E3%82%AB%E3%83%86%E3%82%B4%E3%83%AA%E5%A4%89%E6%95%B0%E3%81%AE,%E3%81%A5%E3%81%91%E3%81%8C%E8%A1%8C%E3%82%8F%E3%82%8C%E3%81%BE%E3%81%99%E3%80%82
enc = OrdinalEncoder()
enc.fit_transform(df['f_27'].str.get(1).values.reshape(-1,1))

In [ ]:
df['f_27']

In [ ]:
enc = OrdinalEncoder()
def feature_eng(data):
    data=train.copy()
    data['char_unique']=data['f_27'].apply(lambda x: len(set(x)))
    for i in range(data.f_27.str.len().max()):
        data['f_27_char{}'.format(i+1)]=enc.fit_transform(data['f_27'].str.get(i).values.reshape(-1,1))
    return data.drop(['f_27'],axis=1)

train_df=feature_eng(data=train)
test_df=feature_eng(data=test)

In [ ]:
test_df.shape

In [ ]:
train_df

In [ ]:
#Frequency
char=train['f_27'].value_counts().nlargest(20)
fig = go.Figure()
fig.add_trace(go.Bar(x=char.index, y=char,
                     hovertemplate='String: %{x}, Frequency: %{y}',
                     showlegend=False))
fig.show()

In [ ]:
train[['f_27']]


In [ ]:
df=train[['f_27']]
for letter in string.ascii_uppercase:
    df['{}'.format(letter)]=df['f_27'].str.count(letter)
df_sum=df.iloc[:,1:].sum(axis=0).rename('sum').reset_index()
pal=sns.color_palette("Spectral_r",28).as_hex()

In [ ]:
string.ascii_uppercase

In [ ]:
df['f_27'].str.count(letter)

In [ ]:
df=train[['f_27']]
for letter in string.ascii_uppercase:
    df['{}'.format(letter)]=df['f_27'].str.count(letter)

In [ ]:
df

In [ ]:
df_sum=df.iloc[:,1:].sum(axis=0).rename('sum').reset_index()
df_sum

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_sum['index'], y=df_sum['sum'],
                     hovertemplate='Letter: %{x}, Frequency: %{y}',
                     showlegend=False))
fig.show()

# neural net work

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, InputLayer, Add
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import metrics, regularizers
from tensorflow.keras.utils import plot_model

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


In [ ]:
test_df.shape

In [ ]:
scaler = StandardScaler()

y=train_df['target']
X=train_df.drop(['target'], axis=1)
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X_test=pd.DataFrame(scaler.transform(test_df.drop(['target'], axis=1)))
#test_df.drop(['target'],


In [ ]:
X_test.shape

In [ ]:
train_df['target']

In [ ]:
test_df.shape

In [ ]:
train_df.shape

In [ ]:
def nn_model():
    
    x_input = Input(shape=(X.shape[1]))
    x = Dense(512, kernel_regularizer=regularizers.l2(1e-5),
              activation='swish')(x_input)
    x = Dense(384, kernel_regularizer=regularizers.l2(1e-5),
              activation='swish')(x)
    x = Dense(256, kernel_regularizer=regularizers.l2(1e-5),
              activation='swish')(x)
    x = Dense(128, kernel_regularizer=regularizers.l2(1e-5),
              activation='swish')(x)
    x = Dense(64, kernel_regularizer=regularizers.l2(1e-5),
              activation='swish')(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=x_input, outputs=output)
    
    return model

model = nn_model()
plot_model(model, show_layer_names=False, show_shapes=True)

In [ ]:
X_test.shape

In [ ]:
y_valid, nn_val_preds, nn_test_preds=[],[],[]
cal_true, cal_pred=[],[]
k_fold = KFold(n_splits=5, shuffle=True, random_state=21)


In [ ]:
#X,y 
for fold, (train_idx, val_idx) in enumerate(k_fold.split(X, y)):
    print(train_idx.shape)
    print(X.iloc[train_idx,:].shape)
    print(val_idx.shape)
    

In [ ]:
for fold, (train_idx, val_idx) in enumerate(k_fold.split(X, y)):
    
    print("\n*****Fold {}*****".format(fold+1))
    X_train, y_train = X.iloc[train_idx,:], y[train_idx]
    X_val, y_val = X.iloc[val_idx,:], y[val_idx]
    print("Train shape: {}, {}, Valid shape: {}, {}".format(
        X_train.shape, y_train.shape, X_val.shape, y_val.shape))

In [ ]:
y.shape

In [ ]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# with tpu_strategy.scope():
#call backs
model = nn_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics=[metrics.AUC(name = 'auc')])

lr = ReduceLROnPlateau(monitor='val_auc', factor=0.5,  patience=3, verbose=True)

es = EarlyStopping(monitor='val_auc', mode='max', patience=5, 
                           restore_best_weights=True, verbose=True)
        
model.fit(X_train, y_train,
                  validation_data=(X_val, y_val), 
                  epochs=20, batch_size=4096, 
                  callbacks=[es,lr], verbose=True, shuffle=True)
        #call backs https://keras.io/ja/callbacks/#:~:text=%E3%82%B3%E3%83%BC%E3%83%AB%E3%83%90%E3%83%83%E3%82%AF%E3%81%AF%E8%A8%93%E7%B7%B4%E4%B8%AD,%E6%B8%A1%E3%81%99%E3%81%93%E3%81%A8%E3%81%8C%E3%81%A7%E3%81%8D%E3%81%BE%E3%81%99%EF%BC%8E

In [ ]:
X_test.shape

NumPy配列ndarrayのサイズ1の次元を削除するnp.squeeze()

In [ ]:
nn_preds = model.predict(X_val).squeeze()

In [ ]:
nn_preds

In [ ]:
np.mean(nn_test_preds, axis=0)

In [ ]:
y_valid.append(y_val)
nn_val_preds.append(nn_preds)
nn_test_preds.append(model.predict(X_test).squeeze())

In [ ]:
sub_nn=sub.copy()
sub_nn['target']=np.mean(nn_test_preds, axis=0)

In [ ]:
sub_nn['target'].shape

In [ ]:
np.mean(nn_test_preds, axis=0).shape

In [ ]:
nn_test_preds

In [ ]:
sub_nn=sub.copy()
sub_nn['target']=np.mean(nn_test_preds, axis=0)
sub_nn.to_csv("sub_nn.csv", index=False)
plot_target_predictions(sub_nn)